In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
import math
import multiprocessing as mp
from Neural_Decoding.preprocessing_funcs import get_spikes_with_history
matplotlib.rc('font', size=18)
import scipy
from sklearn.model_selection import KFold
from Area2_analysis.multi_area_funcs import process_train_test
from Area2_analysis.lr_funcs import get_sses_pred, get_sses_mean
from Neural_Decoding.decoders import DenseNNDecoder

In [ ]:
foldername = "/Users/sherryan/area2_population_analysis/random-target/"
import scipy.io as sio
data = sio.loadmat(foldername+'s1_data_trialized.mat')

In [ ]:
acceleration = data['acceleration']
velocity = data['velocity']
position = data['position']
neural_data = data['neural_data']
print(acceleration.shape,acceleration[0,0].shape,'kinematics')
print(neural_data.shape,neural_data[0,0].shape,'neural data')

In [ ]:
n_trials = len(neural_data)
print(n_trials,'trials')
n_neurons = neural_data[0,0].shape[1]
print(n_neurons,'neurons')

In [ ]:
from Area2_analysis.multi_area_funcs import smooth_spk
#original data has 0.01 s = 10 ms bins 
dt = 0.01
gauss_width = 40 #in ms
bin_width = dt*1000

neural_smth_40 = []
for i in range(n_trials):
    neural_d = neural_data[i,0]
    smth_40 = smooth_spk(neural_d, gauss_width, bin_width)
    neural_smth_40.append(smth_40)
print(len(neural_smth_40),neural_smth_40[0].shape,'neural data')

In [ ]:
plt.plot(neural_data[43,0])
plt.show()
plt.plot(neural_smth_40[43])

In [ ]:
counter = 0
for i in range(len(velocity)):
    counter+=velocity[i,0].shape[0]
print(counter)

In [ ]:
from Area2_analysis.lr_funcs import nans
def all_concat_lagged_neural(x, lag_idx):
    x_out = nans([counter,n_neurons])
    trial_save_idx = 0
    for tr in range(len(x)):
        trial_data = x[tr]
        start, end = 20+lag_idx, len(trial_data)-80+lag_idx
        n = len(trial_data[start:end,:])
        x_out[trial_save_idx:trial_save_idx+n,:] = trial_data[start:end,:]
        trial_save_idx+=n
    return x_out

In [ ]:
def flat_kin(y):
    y_flat = nans([counter,2])
    trial_save_idx = 0
    for tr in range(n_trials):
        n = len(y[tr,0])
        y_flat[trial_save_idx:trial_save_idx+n,:] = y[tr,0]
        trial_save_idx+=n
    return y_flat

In [ ]:
invalid_nrn = np.nan 
for i in range(len(lag_axis)):
    x_flat = all_concat_lagged_neural(x, i*2)[:50000]
    x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
    if np.isnan(invalid_nrn).all():
        invalid_nrn = np.isnan(x_proc).any(axis=0)
    else:
        invalid_nrn = invalid_nrn & (np.isnan(x_proc).any(axis=0))

In [ ]:
np.sum(invalid_nrn)

In [ ]:
from Neural_Decoding.metrics import get_R2
from Neural_Decoding.metrics import get_rho

lag_axis = np.arange(-300, 300, 20)
dim = n_neurons
x = neural_smth_40
y = velocity
y_flat = flat_kin(y)
y_proc = y_flat - np.nanmean(y_flat,axis=0)

r2_array = nans([len(lag_axis)])
r_array = nans([len(lag_axis)])
coef_array = nans([len(lag_axis),2,dim])
for i in range(len(lag_axis)):
    x_flat = all_concat_lagged_neural(x, i*2)
    # x_flat = x_flat[:,~invalid_nrn]
    # dnn_all = DenseNNDecoder(units=400,dropout=0.25,num_epochs=10)
    lr_all = GridSearchCV(Ridge(),{'alpha':np.logspace(-4,1,6)})
    x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
    lr_all.fit(x_proc, y_proc)
    # #Get predictions
    # y_pred=lr_all.predict(x_proc)
    # #Get metric of fit
    # r2_array[i]=np.mean(get_R2(y_proc,y_pred))
    # r_array[i]=np.mean(get_rho(y_proc,y_pred)) 
    coef_array[i,:,:] = lr_all.best_estimator_.coef_

    kf = KFold(n_splits=5, shuffle=False)
    true_concat = nans([counter,2])
    pred_concat = nans([counter,2])
    save_idx = 0
    for training_set, test_set in kf.split(range(0,len(x_flat))):
        X_train, X_test, y_train, y_test = process_train_test(x_flat,y_flat,training_set,test_set)
        lr = Ridge(alpha=lr_all.best_params_['alpha'])
        lr.fit(X_train, y_train)
        # dnn = DenseNNDecoder(units=400,dropout=0.25,num_epochs=10)
        # dnn.fit(X_train, y_train)
        y_test_predicted = lr.predict(X_test)
        n = y_test_predicted.shape[0]
        true_concat[save_idx:save_idx+n,:] = y_test
        pred_concat[save_idx:save_idx+n,:] = y_test_predicted
        save_idx += n
    sses =get_sses_pred(true_concat,pred_concat)
    sses_mean=get_sses_mean(true_concat)
    r2 =1-np.sum(sses)/np.sum(sses_mean)     
    r = np.mean([scipy.stats.spearmanr(pred_concat[:,0], true_concat[:,0]).correlation, scipy.stats.spearmanr(pred_concat[:,1], true_concat[:,1]).correlation])
    r2_array[i] = r2; r_array[i] = r;

In [ ]:
idx_max = np.argmax(r2_array)
x_flat = all_concat_lagged_neural(x, idx_max*2)
# x_flat = x_flat[:,~invalid_nrn]
best_decoder = Ridge(alpha=lr_all.best_params_['alpha'])
x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
best_decoder.fit(x_proc, y_proc)
gen_r2_array = nans([len(lag_axis)])
for i in range(len(lag_axis)):
    x_flat = all_concat_lagged_neural(x, i*2)
    x_flat = x_flat[:,~invalid_nrn]
    x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
    y_pred=best_decoder.predict(x_proc)
    #Get metric of fit
    gen_r2_array[i]=np.mean(get_R2(y_proc,y_pred))

In [ ]:
plt.plot(lag_axis,r2_array, color = 'k', label = 'Original')
plt.plot(lag_axis,gen_r2_array, color = 'r', label = 'Best decoder')
plt.axvline(lag_axis[idx_max],color = 'k',linestyle='--')
plt.ylim([0,1])
plt.legend()
plt.title('RT-v, LR')
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')

In [ ]:
plt.plot(lag_axis,r_array)
plt.xlabel('Time lag (ms)')
plt.ylabel('r')

In [ ]:
from Area2_analysis.lr_funcs import nans, angle_between

idx_max = np.argmax(r2_array)

ang_to_max_x = nans([len(lag_axis)])
ang_to_max_y = nans([len(lag_axis)])
for i in range(0, len(coef_array)):
    ang_to_max_x[i] = math.degrees(angle_between(coef_array[i,0,:],coef_array[idx_max,0,:]))
    ang_to_max_y[i] = math.degrees(angle_between(coef_array[i,1,:],coef_array[idx_max,1,:]))
fig, ax = plt.subplots()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
# plt.ylim([-5, 130])
plt.xlim([-310, 310])
plt.scatter(lag_axis, ang_to_max_x,label = 'x-axis',color = 'green')
plt.scatter(lag_axis, ang_to_max_y,label = 'y-axis',color = 'blue')
plt.legend()
plt.xlabel('Time lag (ms)')
plt.ylabel('Angle (degrees)')
mean = np.mean([ang_to_max_x[idx_max], ang_to_max_y[idx_max]])
print(mean)
# plt.vlines(lag_axis[idx_max],-5, mean, color = 'k',linestyle="dashed")
# plt.hlines(mean, -310, lag_axis[idx_max], color = 'k',linestyle="dashed")
plt.tight_layout()
# plt.savefig(figDir + monkey + label + str(0) +'_angle.pdf', dpi = 'figure')
plt.show()

In [ ]:
i = np.argmax(r2_array)
print(lag_axis[i],'ms')
# x_flat = all_concat_lagged_neural(x, i)
# lr_best = GridSearchCV(Ridge(),{'alpha':np.logspace(-4,4,9)})
# x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
# lr_best.fit(x_proc, y_proc)
# pred = lr_best.predict(x_proc)

In [ ]:
plt.plot(lag_axis,r2_array)
plt.axvline(lag_axis[i])
plt.axvline(100)
plt.xlabel('Time lag (ms)')
plt.ylabel('R2')

In [ ]:
plt.plot(y_proc[:1000,0])
plt.plot(pred[:1000,0])
plt.show()
plt.plot(y_proc[:,0])
plt.plot(pred[:,0])

In [ ]:
i = np.argwhere(lag_axis==80)[0,0]
print(lag_axis[i],'ms')
x_flat = all_concat_lagged_neural(x, i)
lr_best = GridSearchCV(Ridge(),{'alpha':np.logspace(-4,4,9)})
x_proc = (x_flat - np.nanmean(x_flat,axis=0))/np.nanstd(x_flat,axis=0)
lr_best.fit(x_proc, y_proc)
pred = lr_best.predict(x_proc)

In [ ]:
plt.plot(y_proc[:1000,0])
plt.plot(pred[:1000,0])
plt.show()
plt.plot(y_proc[:,0])
plt.plot(pred[:,0])